In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions  import from_unixtime
from pyspark.sql.functions  import to_date
from pyspark.sql import Row
from pyspark.sql.functions import to_json, struct
from pyspark.sql import functions as F

In [ ]:
#Creating the schema for the vehicle data json structure
jsonschema = StructType() \
.add("id", StringType()) \
.add("timestamp", TimestampType()) \
.add("rpm", IntegerType()) \
.add("speed", IntegerType()) \
.add("kms", IntegerType()) 

In [ ]:
TOPIC = "kafkaenabledhub"
BOOTSTRAP_SERVERS = "kafkaenabledeventhubns.servicebus.windows.net:9093"
EH_SASL = "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"Endpoint=sb://kafkaenabledeventhubns.servicebus.windows.net/;SharedAccessKeyName=sendreceivekafka;SharedAccessKey=zzzzzzzz\";"
GROUP_ID = "$Default"

# // Read stream using Spark SQL (structured streaming)
# // consider adding .option("startingOffsets", "earliest") to read earliest available offset during testing
kafkaDF = spark.readStream \
    .format("kafka") \
    .option("subscribe", TOPIC) \
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.jaas.config", EH_SASL) \
    .option("kafka.request.timeout.ms", "60000") \
    .option("kafka.session.timeout.ms", "60000") \
    .option("kafka.group.id", "POC") \
    .option("failOnDataLoss", "false") \
    .option("startingOffsets", "latest") \
    .load()




In [ ]:
#Converting binary datatype to string for the dataframe columns. Without this you cannot use from_json function as it expects the column datatype as string not binary
kafkaDF=kafkaDF.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
#Adding new column vehiclejson which is a struct and has 5 columns id, timestamp,rpm,speed and kms
newkafkaDF=kafkaDF.withColumn('vehiclejson', from_json(col('value'),schema=jsonschema))

In [ ]:
kafkajsonDF=newkafkaDF.select("key","value", "vehiclejson.*")

In [ ]:
#you can uncomment and run the below command to view the column values
# display(kafkajsonDF)

### Default (No Trigger option specified)

In [ ]:
#Writing the streaming data to Delta tables with default trigger i.e. no trigger option is specified

kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "dbfs:/Vehiclechkpointkafkaeventhub_Demo_trigger/") \
.option("mergeSchema", "true") \
.start("dbfs:/VehiclechkpointKafkaEventHub_Delta_trigger") 

Out[7]: <pyspark.sql.streaming.StreamingQuery at 0x7f118495ae90>

In [ ]:
%sql
-- Creating the table on delta location
CREATE TABLE IF NOT EXISTS VehicleDetails_KafkaEnabledEventHub_Delta_Trigger
USING DELTA
LOCATION "dbfs:/VehiclechkpointKafkaEventHub_Delta_trigger/"

In [ ]:
%sql
-- You can keep running this cell continuously  and you will find the numbers growing as data is processed as it arrives.
select count(*) from 
VehicleDetails_KafkaEnabledEventHub_Delta_Trigger

count(1)
2800


### Fixed Interval (trigger(processingTime = duration))

In [ ]:
#Cancel the existing write stream query and run the below code
kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "dbfs:/Vehiclechkpointkafkaeventhub_Demo_trigger/") \
.option("mergeSchema", "true") \
.trigger(processingTime='30 seconds') \
.start("dbfs:/VehiclechkpointKafkaEventHub_Delta_trigger") 


Out[8]: <pyspark.sql.streaming.StreamingQuery at 0x7f11849327d0>

In [ ]:
%sql
-- You can keep running this cell continuously and you will find the numbers don't change continuously , it gets updates after every 30 seconds.
select count(*) from 
VehicleDetails_KafkaEnabledEventHub_Delta_Trigger

count(1)
3526


### continuous (trigger(continuous='1 second'))
- Supported Sources
  - Kafka source: All options are supported.
  - Rate source: Good for testing. Only options that are supported in the continuous mode are numPartitions and rowsPerSecond.

- Supported Sinks
  - Kafka sink: All options are supported.
  - Memory sink: Good for debugging.
  - Console sink: Good for debugging. All options are supported. Note that the console will print every checkpoint interval that you have specified in the continuous trigger.

In [ ]:
#Cancel the existing write stream and run the below code. We are writing to console but you can change the snink to Kafka or Memory
kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("console") \
.outputMode("append") \
.trigger(continuous='1 second') \
.start()


Out[8]: <pyspark.sql.streaming.StreamingQuery at 0x7f961860f390>

### Trigger Once (trigger(once=true))

In [ ]:
#Cancel the existing write stream and run the below code. 
#This code will read the stream once since last checkpoint and will stop executing.
kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "dbfs:/Vehiclechkpointkafkaeventhub_Demo_trigger/") \
.option("mergeSchema", "true") \
.trigger(once=True) \
.start("dbfs:/VehiclechkpointKafkaEventHub_Delta_trigger") 


Out[13]: <pyspark.sql.streaming.StreamingQuery at 0x7f11848955d0>

In [ ]:
%sql
-- You can keep running this cell continuously and you will find the numbers don't change continuously , it gets updates after you rerun the write-stream query.
select count(*) from 
VehicleDetails_KafkaEnabledEventHub_Delta_Trigger

count(1)
3745
